In [188]:
#generate random number of restaurants

import itertools
import math
import random


NUM_RESTAURANTS = 10

#dictionary with restaurants 
#key: restaurant id
#value: coordinates
restaurants = {restaurant_id : (random.randint(0, 10), random.randint(0, 10)) for restaurant_id in range(NUM_RESTAURANTS)}

print(restaurants)

{0: (0, 4), 1: (6, 9), 2: (4, 7), 3: (7, 5), 4: (6, 1), 5: (8, 2), 6: (3, 4), 7: (6, 2), 8: (6, 5), 9: (7, 2)}


In [189]:
#generate random number of requests

NUM_REQUESTS = 100

#dictionary with requests
#key: request id
#value: restaurant
requests = {request_id : (random.randint(0, NUM_RESTAURANTS-1)) for request_id in range(NUM_REQUESTS)}

print(requests)

{0: 8, 1: 7, 2: 8, 3: 4, 4: 6, 5: 7, 6: 3, 7: 2, 8: 2, 9: 4, 10: 8, 11: 0, 12: 3, 13: 9, 14: 2, 15: 3, 16: 2, 17: 8, 18: 5, 19: 1, 20: 4, 21: 8, 22: 3, 23: 0, 24: 2, 25: 4, 26: 8, 27: 8, 28: 5, 29: 9, 30: 8, 31: 9, 32: 3, 33: 7, 34: 7, 35: 3, 36: 8, 37: 0, 38: 6, 39: 4, 40: 4, 41: 5, 42: 5, 43: 8, 44: 7, 45: 9, 46: 8, 47: 7, 48: 1, 49: 3, 50: 5, 51: 9, 52: 0, 53: 6, 54: 5, 55: 2, 56: 2, 57: 3, 58: 2, 59: 3, 60: 1, 61: 7, 62: 6, 63: 7, 64: 2, 65: 5, 66: 9, 67: 7, 68: 0, 69: 0, 70: 5, 71: 7, 72: 7, 73: 6, 74: 8, 75: 4, 76: 2, 77: 4, 78: 2, 79: 9, 80: 3, 81: 0, 82: 6, 83: 3, 84: 2, 85: 4, 86: 3, 87: 5, 88: 9, 89: 5, 90: 2, 91: 2, 92: 4, 93: 5, 94: 0, 95: 5, 96: 3, 97: 4, 98: 3, 99: 8}


In [190]:
#generate random number of drivers

NUM_DRIVERS = 60

#dictionary with delivery_driver
#key: driver id
#value: coordinates
drivers = {driver_id : (random.randint(0, 10), random.randint(0, 10)) for driver_id in range(NUM_DRIVERS)}

print(drivers)

{0: (3, 0), 1: (1, 6), 2: (8, 10), 3: (0, 10), 4: (8, 4), 5: (4, 6), 6: (1, 7), 7: (1, 1), 8: (1, 8), 9: (7, 2), 10: (3, 5), 11: (9, 8), 12: (2, 3), 13: (9, 1), 14: (3, 9), 15: (6, 9), 16: (0, 2), 17: (8, 0), 18: (9, 8), 19: (6, 10), 20: (6, 6), 21: (10, 2), 22: (6, 0), 23: (1, 8), 24: (9, 4), 25: (9, 10), 26: (9, 10), 27: (8, 4), 28: (8, 1), 29: (4, 7), 30: (4, 5), 31: (0, 3), 32: (10, 10), 33: (9, 9), 34: (6, 4), 35: (2, 2), 36: (4, 2), 37: (7, 1), 38: (3, 0), 39: (10, 1), 40: (3, 4), 41: (2, 10), 42: (1, 7), 43: (7, 8), 44: (1, 5), 45: (8, 9), 46: (4, 6), 47: (1, 6), 48: (10, 4), 49: (7, 2), 50: (0, 10), 51: (6, 2), 52: (8, 2), 53: (8, 3), 54: (8, 3), 55: (6, 1), 56: (5, 6), 57: (10, 2), 58: (2, 6), 59: (6, 0)}


In [194]:
#run hungarian algorithm and get best driver-request assignment
#here an example for the hungarian algorithm: http://software.clapper.org/munkres/
import timeit
from munkres import Munkres, print_matrix
from scipy.optimize import linear_sum_assignment


#compute the distance between two restaurants
#agrs:
#     point_a: coordinates of point a
#     point_b: coordinates of point b
#RETURN: distance between point a and point b
def compute_distance(point_a, point_b):
    x1,y1 = point_a
    x2,y2 = point_b

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  

#create cost matrix where y_axis are drivers x_axis are trips
#agrs:
#     trips: all possible trips
#RETURN: a matrix (i.e. list of lists) [NUM_DRIVERS,NUM_TRIPS]
def create_matrix(drivers, requests):
    
    #drivers id
    drivers_ids = [driver for driver in drivers.keys()]
    
    #we assign an id to each trip by concatenating 
    requests_ids = [request for request in requests.keys()]   
    
    
    driver_restaruants_matrix = []
    for driver_id in drivers_ids:
        row = []
        for request_id in requests_ids:
            
            #cost from driver to first restaruant
            cost_d2r = compute_distance(drivers[driver_id], restaurants[int(requests[request_id])])
            
            #TODO here we should add the distance from restaurant to client
            #for this example it should not matter
            #we will add this in the next example where requests can be (1-2)
            cost_final_clinet = 0
            
            cost_total = cost_d2r + cost_final_clinet
            
            row.append(cost_total)
            
        driver_restaruants_matrix.append(row)
    
    return drivers_ids, requests_ids, driver_restaruants_matrix

    
drivers_ids, trips_ids, cost_matrix = create_matrix(drivers, requests)

#optimize matrix (i.e. Hungarian algorithm)
m = Munkres()



#start time
start = timeit.default_timer()

indexes = m.compute(cost_matrix)

#end time
stop = timeit.default_timer()

total=0
for row, column in indexes:
    value = cost_matrix[row][column]
    total += value
    print("driver {} goes to {}".format(drivers_ids[row], trips_ids[column]))
    
print("total cost: "+str(total))
print('computation time: ', stop - start,' seconds')  


driver 0 goes to 3
driver 1 goes to 11
driver 2 goes to 19
driver 3 goes to 7
driver 4 goes to 86
driver 5 goes to 8
driver 6 goes to 14
driver 7 goes to 94
driver 8 goes to 16
driver 9 goes to 13
driver 10 goes to 4
driver 11 goes to 6
driver 12 goes to 38
driver 13 goes to 93
driver 14 goes to 24
driver 15 goes to 60
driver 16 goes to 37
driver 17 goes to 28
driver 18 goes to 22
driver 19 goes to 48
driver 20 goes to 99
driver 21 goes to 41
driver 22 goes to 92
driver 23 goes to 55
driver 24 goes to 95
driver 25 goes to 98
driver 26 goes to 96
driver 27 goes to 12
driver 28 goes to 42
driver 29 goes to 56
driver 30 goes to 53
driver 31 goes to 52
driver 32 goes to 49
driver 33 goes to 15
driver 34 goes to 2
driver 35 goes to 62
driver 36 goes to 1
driver 37 goes to 97
driver 38 goes to 25
driver 39 goes to 18
driver 40 goes to 73
driver 41 goes to 58
driver 42 goes to 64
driver 43 goes to 59
driver 44 goes to 68
driver 45 goes to 57
driver 46 goes to 76
driver 47 goes to 69
driver 48

In [196]:
#toy example to play with

cost_matrix = [[1,10000,10000],[10000,1,10000],[10000,1000,1]] #total cost should be 3, and result a diagonal
#start time
start = timeit.default_timer()

indexes = m.compute(cost_matrix)

#end time
stop = timeit.default_timer()

total=0
for row, column in indexes:
    value = cost_matrix[row][column]
    total += value
    print("driver {} goes to {}".format(drivers_ids[row], trips_ids[column])) 

print("total cost: "+str(total))
print('computation time: ', stop - start,' seconds')  

driver 0 goes to 0
driver 1 goes to 1
driver 2 goes to 2
total cost: 3
computation time:  0.00013806299830321223  seconds
